# TensorFlow script mode training and serving

Script mode is a training script format for TensorFlow that lets you execute any TensorFlow training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native TensorFlow support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

In addition, this notebook demonstrates how to perform real time inference with the [SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container). The TensorFlow Serving container is the default inference method for script mode. 

# Set up the environment

Let's start by setting up the environment:

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

# Construct a script for distributed training

At the end of the training job we have added a step to export the trained model to the path stored in the environment variable ``SM_MODEL_DIR``, which always points to ``/opt/ml/model``. This is critical because SageMaker uploads all the model artifacts in this folder to S3 at end of training.

Here is the entire script:

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='imagequality',
                       entry_point='imagequality_keras_sm.py',
                       source_dir='imagequality',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 1,'batch-size' : 200},
                       train_instance_count=1, train_instance_type='ml.m5.xlarge')

In [ ]:
# Tensorboard is at https://rekognitioncustomlabel.notebook.ap-southeast-1.sagemaker.aws/proxy/6006/


In [ ]:
estimator.fit({'train' :  's3://waynetoh-ml/ava_dataset/train',
             'validation' :  's3://waynetoh-ml/ava_dataset/validation',
             'eval' :  's3://waynetoh-ml/ava_dataset/eval'})

#Tensorboard is not supported with script mode. You can run the following command: 
#tensorboard --logdir s3://sagemaker-ap-southeast-1-284245693010/cifar10-2020-04-27-02-17-46-611/model --host localhost --port 6006

# Deploy the trained model to an endpoint

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time. We will use the TensorFlow Serving container for the endpoint, because we trained with script mode. This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol. The [Using your own inference code]() document explains how SageMaker runs inference containers.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge',endpoint_type='tensorflow-serving')

# Invoke the endpoint

Let's download the training data and use that as input for inference.

In [ ]:
import numpy as np
import json
from matplotlib.pyplot import imshow, figure
from tensorflow.python.keras.preprocessing.image import load_img
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

aws s3 cp s3://waynetoh-ml/shopee_dataset/keyboard /tmp
    
test_image = load_img("/tmp/cb2a641916ef65c5a1668eb4e5a9be0b.jpg", target_size=(224, 224))
test_image_array = np.array(test_image).reshape((1, 224, 224, 3))
figure()    
imshow(test_image)

scores = predictor.predict(test_image_array)
print score

In [ ]:
import numpy as np
import json
from matplotlib.pyplot import imshow, figure
from tensorflow.python.keras.preprocessing.image import load_img
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

# calculate mean score for AVA dataset
def mean_score(scores):
    si = np.arange(1, 11, 1)
    mean = np.sum(scores * si)
    return mean

# calculate standard deviation of scores for AVA dataset
def std_score(scores):
    si = np.arange(1, 11, 1)
    mean = mean_score(scores)
    std = np.sqrt(np.sum(((si - mean) ** 2) * scores))
    return std


for i in range(3,9):
    
    test_image = load_img("/home/ec2-user/ava_dataset/images/95827" + str(i) +".jpg", target_size=(224, 224))
    test_image_array = np.array(test_image).reshape((1, 224, 224, 3))
    figure()    
    imshow(test_image)

    scores = predictor.predict(test_image_array)
    

    # NIMA model produces a distribution of ratings for any given image, 
    # on a scale of 1 to 10, with 10 being the highest aesthetic score associated to an image.
    # It assigns likelihoods to each of the possible scores, 

    mean = mean_score(scores['predictions'][0])
    std = std_score(scores['predictions'][0])
    
    print("======= image " + str(i) + "=======")
    print(scores['predictions'][0])
    print('Mean score =',mean)
    print('Std score =',std)
    
    print("NIMA Score : %0.3f +- (%0.3f)" % (mean, std))
    
    print ()


In [ ]:
import numpy as np
import json
from matplotlib.pyplot import imshow, figure
from tensorflow.python.keras.preprocessing.image import load_img
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

# calculate mean score for AVA dataset
def mean_score(scores):
    si = np.arange(1, 11, 1)
    mean = np.sum(scores * si)
    return mean

# calculate standard deviation of scores for AVA dataset
def std_score(scores):
    si = np.arange(1, 11, 1)
    mean = mean_score(scores)
    std = np.sqrt(np.sum(((si - mean) ** 2) * scores))
    return std


for i in range(1,10):
    
    test_image = load_img("/home/ec2-user/shopee_dataset/shopee" + str(i) +".png", target_size=(224, 224))
    test_image_array = np.array(test_image).reshape((1, 224, 224, 3))
    figure()    
    imshow(test_image)

    scores = predictor.predict(test_image_array)
    

    # NIMA model produces a distribution of ratings for any given image, 
    # on a scale of 1 to 10, with 10 being the highest aesthetic score associated to an image.
    # It assigns likelihoods to each of the possible scores, 
    # For a given mean valuehowever, images with a high variance seem more likely tobe edgy or subject to interpretation, 
    # while images with a lowvariance tend to use conventional styles or depict conven-tional subject matter. 

    mean = mean_score(scores['predictions'][0])
    std = std_score(scores['predictions'][0])
    
    print("======= image " + str(i) + "=======")
    print(scores['predictions'][0])
    print('Mean score =',mean)
    print('Std score =',std)
    
    print("NIMA Score : %0.3f +- (%0.3f)" % (mean, std))
    
    print ()


Examine the prediction result from the TensorFlow 2.1 model.

# Delete the endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)